In [32]:
import sdv
import pandas as pd

print(sdv.version.public)

1.11.0


In [33]:
# import zipfile
# import os
# zip_path = 'drive-download-20240323T140328Z-001.zip'
# extract_path = os.getcwd()

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)


### Loading the data

In [34]:
#viewing the data we have
train = pd.read_csv('train.csv')


# if  a single df
# data = pd.read_csv('data.csv')

<li> I think this notebook will help in synthesizing our data
<li> Our data should not be divided (i mean we should have a single df that contains (train,test, validation) after we do the split)
<li>Problem : we need to try and find a way of making our hmof ids unique and same (cos this ids .cifs are the ones available , different onece will cause a problem)

In [35]:
train.head()

,hMOF-21761,1.4469189829363254,1.0116009116784799,-1.2196517457139742,5.5258512661635395,-1.8613410854197348
0,hMOF-6005116,1.321756,0.810930,-2.301835,4.730921,-1.781057
1,hMOF-5015132,1.749200,1.178655,-0.678929,6.767228,0.474263
2,hMOF-27705,1.749200,1.178655,-0.754667,7.120283,-2.842239
3,hMOF-5072656,1.446919,1.178655,-1.289789,5.132263,-1.197971
4,hMOF-5076184,1.446919,1.011601,-1.284149,6.075576,-2.642411


In [36]:
train_columns = ['id','lcd', 'pld', 'void_fraction', 'surface_area_m2g','pressure_0.01(bar)']
train.columns = train_columns


In [37]:
train.to_csv('new_train.csv', index=False)

column_names = ['id','lcd', 'pld', 'void_fraction', 'surface_area_m2g','surface_area_m2cm3', 'pressure_0.01(bar)']


### Meta data

In [38]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()

metadata.detect_from_csv(filepath='new_train.csv')

In [39]:
python_dict = metadata.to_dict()

   set  'pii=True' if you want to anonymize that column in your corrections
   


In [40]:
python_dict

{'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1',
 'primary_key': 'id',
 'columns': {'id': {'sdtype': 'id'},
  'lcd': {'sdtype': 'numerical'},
  'pld': {'sdtype': 'numerical'},
  'void_fraction': {'sdtype': 'numerical'},
  'surface_area_m2g': {'sdtype': 'numerical'},
  'pressure_0.01(bar)': {'sdtype': 'numerical'}}}

In [41]:
# #in a case where there is mistake in the auto detection of metadata
# metadata.update_columns(
#     column_names=['age', 'transactions', 'session_length'],
#     sdtype='numerical',
#     computer_representation='Float'
# )

In [42]:
# #setting primary key
# metadata.set_primary_key(column_name='guest_email')

In [43]:
#validating metadata
metadata.validate()

In [44]:
metadata.validate_data(data=train)

In [45]:
from sdv.metadata import SingleTableMetadata

metadata.save_to_json(filepath='my_metadata_v1.json')




In [46]:
metadata = SingleTableMetadata.load_from_json(filepath='my_metadata_v1.json')

### Creating a synthesizer
An SDV synthesizer is an object that we  use to create synthetic data. It learns patterns from the real data and replicates them to generate synthetic data.



In [47]:
#fast method
from sdv.lite import SingleTablePreset

# synthesizer = SingleTablePreset(metadata, name='FAST_ML')
# synthesizer.fit(data)

# synthetic_data = synthesizer.sample(num_rows=10)


from sdv.single_table import GaussianCopulaSynthesizer

# Step 1: Create the synthesizer
synthesizer = GaussianCopulaSynthesizer(metadata)

# Step 2: Train the synthesizer
synthesizer.fit(train)

# Step 3: Generate synthetic data
synthetic_data = synthesizer.sample(num_rows=100)

In [48]:
#evaluating our the data with the real ones
from sdv.evaluation.single_table import run_diagnostic

diagnostic = run_diagnostic(
    real_data=train,
    synthetic_data=synthetic_data,
    metadata=metadata
)

Generating report ...
(2/2) Evaluating Data Structure: : 100%|██████████| 1/1 [00:00<00:00, 541.13it/s]

Overall Score: 100.0%

Properties:
- Data Validity: 100.0%
- Data Structure: 100.0%


In [49]:
#measure the data quality or the statistical similarity between the real and synthetic data.
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    train,
    synthetic_data,
    metadata
)

Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 15/15 [00:00<00:00, 352.15it/s]

Overall Score: 79.75%

Properties:
- Column Shapes: 62.36%
- Column Pair Trends: 97.13%


In [50]:
quality_report.get_details('Column Shapes')

,Column,Metric,Score
0,lcd,KSComplement,0.836678
1,pld,KSComplement,0.566265
2,void_fraction,KSComplement,0.915146
3,surface_area_m2g,KSComplement,0.799088
4,pressure_0.01(bar),KSComplement,0.000861


In [51]:
# #saving the synthesizing model for future use
# from sdv.lite import SingleTablePreset

# synthesizer.save('my_synthesizer.pkl')

# synthesizer = SingleTablePreset.load('my_synthesizer.pkl')